<div dir="rtl">
<p>در این خانه کتابخانه های مورد نیاز فراخوانی شده اند.</p>
</div>

In [1]:
import pandas as pd
import numpy as np

<div dir="rtl">
<p>دیتای مورد نظر به یک pandas dataframe ریخته شده است.</p>
</div>

In [2]:
df = pd.read_csv("BankWages_dataset/BankWages.csv", index_col=0)
df.head()

,job,education,gender,minority
1,manage,15,male,no
2,admin,16,male,no
3,admin,12,female,no
4,admin,8,female,no
5,admin,15,male,no


<div dir="rtl">
<p>ورودی های غیر عددی با روش label encoding به فیچر های عددی تبدیل شده اند.</p>
</div>

In [3]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

categorized_labels = ["job", "gender", "minority"]
for cl in categorized_labels:
    encoded_label = le.fit_transform(df[cl])
    df.drop(cl, axis=1, inplace=True)
    df[cl] = encoded_label

df.head()

,education,job,gender,minority
1,15,2,1,0
2,16,0,1,0
3,12,0,0,0
4,8,0,0,0
5,15,0,1,0


<div dir="rtl">
<p>خانه زیر آماری با توجه به LABEL ها می دهد.</p>
</div>

In [4]:
df.groupby("minority").describe()

education                                                      job  \
             count       mean       std  min   25%   50%   75%   max  count   
minority                                                                      
0            370.0  13.694595  2.942304  8.0  12.0  15.0  16.0  21.0  370.0   
1            104.0  12.769231  2.555142  8.0  12.0  12.0  15.0  19.0  104.0   

                    ...           gender                                     \
              mean  ...  75%  max  count      mean       std  min  25%  50%   
minority            ...                                                       
0         0.470270  ...  1.0  2.0  370.0  0.524324  0.500084  0.0  0.0  1.0   
1         0.201923  ...  0.0  2.0  104.0  0.615385  0.488860  0.0  0.0  1.0   

                    
          75%  max  
minority            
0         1.0  1.0  
1         1.0  1.0  

[2 rows x 24 columns]

<div>
<p>
در این خانه داده های تست و ترین با نسبت ۳۰ به ۷۰ جدا شده اند.
</p>
</div>

In [189]:
from sklearn.model_selection import train_test_split

X = df.drop("minority", axis=1).values
y = df["minority"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

<div>
<p>
پیاده سازی تابع همانی
</p>
</div>

In [224]:
class IdentityFunction:

    def output(self, x):
        return x

    def gradient(self, x):
        return np.ones(len(x))

<div>
<p>
پیاده سازی تابع سیگموید
</p>
</div>

In [225]:
class SigmoidFunction:

    def output(self, x):
        return 1 / (1 + np.exp(-x))

    def gradient(self, x):
        s = self.output(x)
        return s * (1 - s)


<div>
<p>
در این خانه موجودیت نرون به همراه متود های مربوطه تعریف شده است.
</p>
</div>

In [466]:
class Neuron:
    def __init__(self, input_size):
        self.input_size = input_size
        self.w = np.random.random(input_size)
        self.gradient_tape = []

    def zero_gradient(self):
        self.gradient_tape = []
        return True

    def add_gradient(self, value):
        self.gradient_tape.append(value)
        
    def generate_local_gradient(self, coefficient):
        self.gradient_tape[-1] = np.dot(coefficient, self.gradient_tape[-1])

    def call(self, inputs):
        induced_local_field = np.dot(self.w, inputs)
        return induced_local_field

    def update(self, learning_rate, momentum):
        gradient_input = np.ones(self.input_size)
        for g in self.gradient_tape:
            gradient_input = np.dot(gradient_input, g)
        
        print(gradient_input)
#         print(self.gradient_tape)
#         print(gradient_input)
#         self.w = self.w + np.dot(learning_rate, gradient_input)
#         print("#############")
        print(self.w + np.dot(learning_rate, gradient_input))
#         print(self.w)
#         self.w = self.w + np.dot(learning_rate, gradient_input)
#         print(self.w)
                                 
    @property
    def weights(self):
        return self.w

    @property
    def gradient(self):
        return self.gradient_tape

<div>
<p>
در این خانه لایه ای از نرون ها به همراه متد های مربوط تعریف شده است.
</p>
</div>

In [467]:
class NeuralLayer:
    def __init__(self, inputsize_neuroncount, activation_function):
        self.inputsize_neuroncount = inputsize_neuroncount
        self.neurons = [Neuron(inputsize_neuroncount[0]) for _ in range(inputsize_neuroncount[1])]
        self.a_f = activation_function()

    def calculate_outputs(self, x_input):
        # calculate gradient for backward pass
        for neuron in self.neurons:
            neuron.zero_gradient()
            neuron.add_gradient(x_input)
            neuron.add_gradient(self.a_f.gradient(x_input))

        return np.array([self.a_f.output(neuron.call(x_input)) for neuron in self.neurons])
    
    def multiply_local_gradient(self, value):
        for neuron in self.neurons:
            neuron.generate_local_gradient(value)

    def update_weights(self, learning_rate, momentum):
        for neuron in self.neurons:
            neuron.update(learning_rate, momentum)
            
    @property
    def weights(self):
        return [neuron.weights for neuron in self.neurons]
    
    @property
    def neuron_list(self):
        return list(self.neurons)


<div>
<p>
شبکه عصبی از نوع fully connected با متد های مربوطه تعریف شده اند
</p>
</div>

In [468]:
class NNModel:
    def __init__(self, *layers, learning_rate, momentum):
        self.layers = layers
        self.learning_rate = learning_rate
        self.momentum = momentum

    def forward(self, x_input):
        layer_input = x_input
        for layer in self.layers:
            layer_output = layer.calculate_outputs(layer_input)
            layer_input = layer_output

        # output of the neural network is the output of the last layer
        neural_net_output = layer_input
        return neural_net_output
    
    def backward(self, loss):
        
        for i, layer in enumerate(self.layers[::-1]):
            if layer == self.layers[-1]:
                layer.multiply_local_gradient(np.full((layer.inputsize_neuroncount[0]), loss[0]))
            else:
                for j, neuron in enumerate(layer.neuron_list):
                    
                    sum_of_w_in_gradient = np.ones(neuron.input_size)
                    for k, nk in enumerate(self.layers[i-1].neuron_list):
                        sum_of_w_in_gradient += np.dot(nk.weights[j], nk.gradient_tape[-1])
                    neuron.generate_local_gradient(sum_of_w_in_gradient)
                    
    def update_weights(self):
        for layer in self.layers:
            layer.update_weights(self.learning_rate, self.momentum)
                
    def calculate_loss(self, y_predicted, y_true):
        return y_true - y_predicted
        

    def fit(self, x_input, y_output):
        for i in range(len(x_input)):
            output = self.forward(x_input[i])
            loss = self.calculate_loss(output, y_output[i])
            self.backward(loss)
            self.update_weights()
            
    @property
    def weights(self):
        return [neural_layer.weights for neural_layer in self.layers]

<div>
<p>
مدلی از شبکه عصبی ترین می شود.
</p>
</div>

In [469]:
n = NNModel(
    NeuralLayer((3, 3), IdentityFunction),
    NeuralLayer((3, 3), IdentityFunction),
    NeuralLayer((3, 1), IdentityFunction),
    learning_rate=0.1,
    momentum=0
)

